In [2]:
import os,json,csv, re
import opencc
import spacy
from tqdm import tqdm

nlp = spacy.load("zh_core_web_sm")

txt_file = 'article.txt'
csv_file = 'article.csv'
sent_file = 'sents.csv'


2022-12-27 00:18:59.927573: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-27 00:19:00.399053: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-27 00:19:01.942492: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-27 00:19:01.942595: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [3]:
# read .txt

rows = []
with open(txt_file,'r',encoding='utf-8') as f:
    rows = f.readlines()
    for i in range(len(rows)):
        try:
            rows[i] = json.loads(rows[i])
        except:
            pass

In [4]:
# data cleaning

for i in range(len(rows)):
    try:
        text = rows[i]['content']
        text = re.sub(pattern="[&].*?[;]",repl='',string=text)  
        text = re.sub(pattern="[<u>].*?[</u>]",repl='',string=text)
        text = re.sub(pattern="[<].*?[>]",repl='',string=text)  
        text = re.sub(pattern="[https://].*?[/]",repl='',string=text)  
        text = text.replace('作文/Www.Z','')
        text = text.replace('\n','')

        # text = converter.convert(text)
        rows[i]['content'] = text
    except:
        pass

In [5]:
# save to csv

labels = list(rows[0].keys())

with open(csv_file, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=labels)
    writer.writeheader()
    for row in rows:
        try:
            writer.writerow(row)
        except:
            pass


In [6]:
# save to cents.csv

line_counts = 0

with open(sent_file, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['content'])
    writer.writeheader()
    for r in tqdm(range(0,20000)):
        row = rows[r]
        doc = nlp(row['content'])
        sents = [sent for sent in doc.sents]
        for i in sents:
            try:
                writer.writerow({'content':i})
            except:
                pass

            line_counts += 1

print('lines:',line_counts)


100%|██████████| 20000/20000 [11:39<00:00, 28.60it/s] 

lines: 416330
